In [1]:
import simpy
import random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

class Persona:
    def __init__(self, env, id, infect_prob, rec_time):
        self.env = env
        self.id = id
        self.estado = "S"
        self.infect_prob = infect_prob
        self.rec_time = rec_time
        self.en_proceso = False
        self.historial = []

    def infectar(self):
        if self.estado == "S" and not self.en_proceso:
            self.en_proceso = True
            if random.random() < self.infect_prob:
                self.estado = "I"
                yield self.env.timeout(self.rec_time)
                self.estado = "R"
            self.en_proceso = False

def simulacion(env, nombre, infect, infect_prob, rec_time, num_personas=100, duracion=30):
    personas = [Persona(env, i, infect_prob, rec_time) for i in range(num_personas)]
    personas[0].estado = "I"

    tiempos = []
    susceptibles = []
    infectados = []
    recuperados = []

    while env.now < duracion:
        for persona in personas:
            persona.historial.append((env.now, persona.estado))

        for persona in personas:
            if persona.estado == "I":
                vecinos = random.sample(personas, infect)
                for vecino in vecinos:
                    env.process(vecino.infectar())

        s = sum(p.estado == "S" for p in personas)
        i = sum(p.estado == "I" for p in personas)
        r = sum(p.estado == "R" for p in personas)

        tiempos.append(env.now)
        susceptibles.append(s)
        infectados.append(i)
        recuperados.append(r)

        yield env.timeout(1)

    # ========== Gráfica SIR ==========
    plt.figure(figsize=(8, 5))
    plt.plot(tiempos, susceptibles, label="Susceptibles", color="blue")
    plt.plot(tiempos, infectados, label="Infectados", color="red")
    plt.plot(tiempos, recuperados, label="Recuperados", color="green")
    plt.xlabel("Tiempo")
    plt.ylabel("Número de personas")
    plt.title(f"Curva epidémica (SIR) - {nombre}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Marcar pico de infección
    pico_idx = max(range(len(infectados)), key=lambda idx: infectados[idx])
    tiempo_pico = tiempos[pico_idx]
    valor_pico = infectados[pico_idx]

    plt.scatter([tiempo_pico], [valor_pico], color="black", s=50, zorder=5)
    plt.annotate(
        f"Pico: t={tiempo_pico}, I={valor_pico}",
        xy=(tiempo_pico, valor_pico),
        xytext=(tiempo_pico + 1, valor_pico + 5),
        arrowprops=dict(arrowstyle="->", color="black")
    )

    # Mostrar R0
    r0 = infect * infect_prob * rec_time
    plt.figtext(0.5, -0.05, f"{nombre} - Número Básico de Reproducción (R₀): {r0:.2f}",
                wrap=True, horizontalalignment='center', fontsize=10)
    plt.subplots_adjust(bottom=0.2)
    plt.show()

    return personas, duracion

# ========== Escenarios ==========
escenarios = [
    {"nombre": "Escenario 1", "infect": 5, "infect_prob": 0.5, "rec_time": 14},
    {"nombre": "Escenario 2", "infect": 4, "infect_prob": 0.2, "rec_time": 19},
]

# ========== Ejecutar simulaciones ==========
resultados = []
for escenario in escenarios:
    print(f"\nSimulando: {escenario['nombre']}")
    random.seed(42)
    env = simpy.Environment()
    personas, duracion = simulacion(
        env,
        escenario["nombre"],
        escenario["infect"],
        escenario["infect_prob"],
        escenario["rec_time"],
        num_personas=50,
        duracion=60
    )
    resultados.append((escenario["nombre"], personas, duracion))

# ========== Gráfico de comportamiento individual ==========
estado_color = {"S": "blue", "I": "red", "R": "green"}

for nombre, personas, duracion in resultados:
    plt.figure(figsize=(10, 6))
    for i, persona in enumerate(personas):
        historial = persona.historial
        for t in range(1, len(historial)):
            tiempo_inicial = historial[t - 1][0]
            tiempo_final = historial[t][0]
            estado = historial[t - 1][1]
            plt.hlines(i, tiempo_inicial, tiempo_final, colors=estado_color[estado], linewidth=2)

        # último estado hasta el final
        if historial:
            ult_tiempo, ult_estado = historial[-1]
            plt.hlines(i, ult_tiempo, duracion, colors=estado_color[ult_estado], linewidth=2)

    plt.xlabel("Tiempo")
    plt.ylabel("Individuos")
    plt.title(f"Evolución de estados individuales - {nombre}")
    plt.grid(True)
    legend_handles = [
        mpatches.Patch(color=color, label=label)
        for label, color in estado_color.items()
    ]
    plt.legend(handles=legend_handles, loc='upper right')
    plt.tight_layout()
    plt.show()



Simulando: Escenario 1


ValueError: too many values to unpack (expected 2)